<a href="https://colab.research.google.com/github/thill08450/MTH4332-Assignment-3/blob/main/MTH4332/IsingModel/IsingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run this cell first

In [ ]:
# this code enables the automated feedback. If you remove this, you won't get any feedback
# so don't delete this cell!
try:
  import AutoFeedback
except (ModuleNotFoundError, ImportError):
  !pip install AutoFeedback
  import AutoFeedback

try:
  from testsrc import test_main
except (ModuleNotFoundError, ImportError):
  !pip install "git+https://github.com/autofeedback-exercises/exercises.git@main#subdirectory=New-MTH4332/IsingModel"
  from testsrc import test_main

def runtest(tlist):
  import unittest
  from contextlib import redirect_stderr
  from os import devnull
  with redirect_stderr(open(devnull, 'w')):
    suite = unittest.TestSuite()
    for tname in tlist:
      suite.addTest(eval(f"test_main.UnitTests.{tname}"))
    runner = unittest.TextTestRunner()
    try:
      runner.run(suite)
    except AssertionError:
      pass


# The 2D Ising Model

In the example report I used Monte Carlo simulation to investigate the properties of the 2D Ising model.  The following exercises will help you to reproduce the calculations that I performed.  As always you should start these exercises by running the following cell which loads some python libraries we will use in the remaining exercises.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

You should then watch the following video, which explains how we can construct a representation for the state of the Ising model using a 2D NumPy array.

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/hXqYslTwYKU?si=ZKV1w5FRd5XrncNO" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

# Generating a microstate

As the video explained, the 2D Ising model can be used to describe interacting spins on a two-dimensional square lattice. Each spin can be in one of two states, up (+1) or down (-1) so we can thus represent
the microstate using a 2D NumPy array with elements that are either 1 or -1.

We can create a 2D NumPy array full of zeros with 5 rows and 5 columns using the command:

```python
myarray = np.zeros([5,5])
```

To assign value to element (i,j) of this array you use the command:

```python
myarray[i,j] = 1
```

and to loop over all the elements of the array you can use a double loop like this:

```python
for i in range(5) :
    for j in range(5) : print( myarray[i,j] )
```

(This double loops prints all the elements in the array)

Your task in this exercise is to write a function called `getstate` that takes one argument `N`.  Your function should return an array that contains a randomly selected microstate for a 2D Ising model
composed of (N X N) spins.  You should generate your state so that approximately half of the spins will be in the up (+1) state and half should be in the down (-1) state.  Your function should not return
the same state every time it is called, however.  There should be some randomness in the way the spins are assigned.


In [ ]:
def getstate(N) :
    arr = np.random.rand(N,N)
    for i in range(N):
      for j in range(N):
        if arr[i,j]>0.5:
          arr[i,j] = 1
        else:
          arr[i,j] = -1
    return arr

# You don't need to adjust this code.  It is just here
# so you can look at what your function is generating.
# This will generate a state for a 4x4 lattice of spins
print( 'Random 4x4 state', getstate(4) )
# This will generate a state for a 5x4 lattice of spins
print( 'Random 5x5 state', getstate(5) )

In [ ]:
runtest(['test_estimate'])

# Flipping a randomly selected spin

The Monte Carlo algorithm that we are going to implement works by making random moves to new microstates of the system. In this exercise we are thus
going to write code to generate a new microtates.

To complete this exercise you need to write four functions.

The first of these functions is called `flipSpin` and takes three arguments.  The first of these arguments `spins` is a 2D
NumPy array that contains the current microstate.  The second and third arguments `i` and `j` are then the indices of the indices for the spin
coordinate that should be flipped.  The first of these indices is the first index of the coordinate that should be flipped and the second is the second.
This function should return a NumPy array that contains the microstate with the flipped coordinate.

The second of these functions is called `flipAllSpins`.  This function takes a single argument called `spins`, which is a 2D
NumPy array that contains the current microstate.  The function should return a new microstate in which every spin has flipped direction.

The third of these function is called `chooseMove`.  This function randomly selects what type of move to perform.  It takes a single argument, `spins`, in input, which
is a 2D NumPy array that contains the current microstate.  This function should return a Bernoulli random variable with:

$$
p = \frac{1}{1+N^2}
$$

Where $N^2$ is the number of spins in the lattice.   If this random variable is one it tells your Monte Carlo to generate a new structure by flipping all the spins.

The final function you need to write is called `chooseSpin`.  This function chooses a spin to flip.  It again takes a single argument, called `spins`, in input, which
is a 2D NumPy array that contains the current microstate.  It should return a single number that specify a the x-coordinate of the spin that should be flipped.

N.B. When you come to writing your Monte Carlo code you will combine all these functions into a single function for generating moves. Your code will:

1. Use functionality like that in `chooseMove` to generate a Bernoulli random variable X that determines whether you flip a single spin or all the spins.

2. If X=1 then you call `flipAllSpins` to flip all the spins and you are done.

4. If X=0 then you call `chooseSpin` twice to select the spin to flip and then pass the two coordinates that are output to `flipSpin` to generate the new microstate.

I have only asked you to write these functiosn separately here so that I can more easily write tests to ensure that all the parts of your implementation are correct.


In [ ]:
def flipSpin( spins, i, j ) :
    spins[i,j] = spins[i,j]*-1
    return spins


def flipAllSpins( spins ) :
  N,M = spins.shape
  for i in range(N):
    for j in range(M):
      spins[i,j] = spins[i,j]*-1
  return spins



def chooseMove( spins ) :
    N = len(spins)
    p = 1 / (1 + N**2)
    return np.random.binomial(1, p)



def chooseSpin( spins ) :
    N = len(spins)
    choice = np.random.randint(0, N, dtype = int)
    return choice

In [ ]:
runtest(['test_flipSpin', 'test_flipAllSpins', 'test_chooseMove', 'test_chooseSpin'])

# Calculating the magnetisation

The average magnetisation per spin for a particular configuration is calculated as:

$$
M = \frac{1}{N} \sum_{i=1}^N s_i
$$

where the sum runs over all of the $N$ spins in the lattice and the $s_i$ values are the spin coordinates.  Your task for this exercise is to complete
the function called `magnetisation` and to write a function that takes a 2D NumPy array, `spins`, that contains the spin coordinate for a
microstate of the system and that returns the magnetisation calculated using the formula above.


In [ ]:
def magnetisation( spins ) :
   N,M = spins.shape
   len = M*N
   sum = 0
   for i in range(N):
    for j in range(M):
      sum += spins[i,j]
   mag = sum/len

   return mag



# The rest of this code is just to check if you code is doing something sensible
spins = np.ones([10,10])
print('The magnetisation of the all up state is', magnetisation( spins ) )
spins = -1*spins
print('The magnetisation of the all down state is', magnetisation( spins ) )


In [ ]:
runtest(['test_mag'])

# Hamiltonian for non-interacting spins

In order to do Monte Carlo we are going to need a Hamiltonian to calculate the energy of the microstates.
In these next three exercies we are thus going to write functions that describe Hamiltonians that become
progressively more and more complicated.  Before you do those exercises watch the following video that will introduce you to the various Hamiltonians we will be using here.

If you want to read more about the Ising model you can a basic introduction [here](https://en.wikipedia.org/wiki/Square_lattice_Ising_model) and a more complicated introduction [here](https://www.thphys.uni-heidelberg.de/~wolschin/statsem20_3s.pdf)

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/6W881fbHGlU?si=m49Y39GiHj3IFXUN" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

The first simple Hamiltonian that you are compute gives the energy as:

$$
E = - H \sum_{i=1}^N s_i
$$

Where $H$ is the magnetic field strength, the sum runs over the $N$ spins in the system and $s_i$ is the spin of the $i$th particle.  You will notice that this is the Hamiltonian for a set of non-interacting spins that you studied for the first of the assignments in this module.  If the spins are not interacting, the way they are
arranged is not particularly important so the 2D model of the spins is equivalent to the 1D model we have studied before.

To complete this exercise you need to write a function called `hamiltonian` that returns the energy calculated using
the formula above.  This function takes two arguments:

* `spins` is a 2D NumPy array that contains the microscopic coordinates of all the spins.
* `H` is the magnetic field strength.

In [ ]:
def hamiltonian( spins, H ) :
   N,M = spins.shape
   sum = 0
   for i in range(N):
    for j in range(M):
      sum += spins[i,j]
   E = sum*(-H)

   return E


# The rest of this code is just to check if you code is doing something sensible
spins = np.ones([10,10])
print('The energy of the all up state is', hamiltonian( spins, -1 ) )
print('The energy of the all up state is', hamiltonian( spins, +1 ) )
spins = -1*spins
print('The energy of the all down state is', hamiltonian( spins, -1 ) )
print('The energy of the all down state is', hamiltonian( spins, +1 ) )


In [ ]:
runtest(['test_hamiltonian1'])

# Hamiltonian for interacting spins

Having revised the Hamiltonian for a set of non-interacting spins lets get on to something more interesting and
write a function that calculate a Hamiltonian that describes the interaction between spins.  We are going to use the following Hamiltonian here:

$$
E = - \sum_{\langle j,j \rangle} s_i s_j
$$

Here the sum runs over all pairs of spins that are adjacent in this lattice and $s_i$ and $s_j$ are spins coordinates.  The following diagram does a better job of describing how the spins in our system interact that the equation.

![](https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/New-MTH4332/IsingModel/spindiagram.png)

You can see that each spin indicates with its four nearest neighour.  So the blue spin in the center of the diagram below interacts with the four spins that are shown in red.  You can implement this Hamiltonian by doing a sum over all the spins in the lattice.  Each spin should be multiplied by the spins of the four atoms that are adjacent to it in the lattice.  The whole sum then needs be divided by two to account for the double counting.

To complete this exercise you need to write a function called `hamiltonian` that returns the energy calculated using the formula above.  This function takes a single argument called `spins`.  This argument is a 2D NumPy array that contains the microscopic coordinates of all the spins.


In [ ]:
def hamiltonian( spins ) :
    # right neighbours
    E = np.sum(spins * np.roll(spins, shift=1, axis=1))
    # down neighbours
    E += np.sum(spins * np.roll(spins, shift=1, axis=0))
    return -E


# The rest of this code is just to check if you code is doing something sensible
spins = np.ones([10,10])
print('The energy of the all up state is', hamiltonian( spins ) )
spins = -1*spins
print('The energy of the all down state is', hamiltonian( spins ) )

In [ ]:
runtest(['test_hamiltonian2'])

# The full Hamiltonian

Lets now combine what we learned in the last two exercises and implement the full Hamiltonian for the 2D Ising model that is given by the equation below:

$$
E = - H \sum_i s_i - \sum_{\langle j,j \rangle} s_i s_j
$$

This is the combination of the interaction with the spins with the magnetic field with strength H and the interactions between the pairs of adjacent spins that we saw in the
previous exercise.

So as with the two previous exercicses, to complete this exercise you need to write a function called `hamiltonian` that returns the energy calculated using
the formula above.  This function takes two arguments:

* `spins` is a 2D NumPy array that contains the microscopic coordinates of all the spins.
* `H` is the magnetic field strength.



In [ ]:
def hamiltonian( spins, H ) :
    term1 = np.sum(spins)
    term2 = np.sum(spins * np.roll(spins, shift=1, axis=1))
    # down neighbours
    term2 += np.sum(spins * np.roll(spins, shift=1, axis=0))
    E = -H*term1 - term2

    return E



# The rest of this code is just to check if you code is doing something sensible
spins = np.ones([10,10])
print('The energy of the all up state is', hamiltonian( spins, -1 ) )
print('The energy of the all up state is', hamiltonian( spins, +1 ) )
spins = -1*spins
print('The energy of the all down state is', hamiltonian( spins, -1 ) )
print('The energy of the all down state is', hamiltonian( spins, +1 ) )


In [ ]:
runtest(['test_hamiltonian3'])

# Running Monte Carlo

The next few exercises are going to explain how to write a Monte Carlo code for simulating the Ising model.  However, before you attempt the exercises, it is worth watching the following video, which gives a brief introduction to the Metropolis Monte Carlo algorithm that we will be using here.

_N.B. When this video talks about moving one of the atoms a small amount you instead use the tricks we have developed above to generate new mirostates for your system of Ising spins._

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/gZQ3AUbjJJc?si=_IZSBGqw-6cUEsI4" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

Before we try to implement a Monte Carlo code for simulating the Ising model we are going to first write a program to sample one of the 1D energy landscapes that you investigated
using MD in the last batch of this exercises with the Monte Carlo algorithm that was discussed in the video.  With that in mind, I have written a function called `energy` in the following cell.  This function takes the position of the particle and returns the energy.  As you can see the energy in this case is just a harmonic potential.

You are going to write code that generates a graph that shows the series of $x$ values that are sampled by the Monte Carlo trajectory.  I will then test that you have implemented Monte Carlo correctly by determining if the series of random variables is sampling from the correct distribution.  I have started the process
of writing this code for you by setting the inital position of the particle equal in the variable `pos`.  You will also see that I have set the variable `oldenergy` equal to the energy that the system has when the particle is at `pos`.

I have then written a loop that will generate `nsamples` random variables using Monte Carlo and created NumPy arrays called `xvals` and `yvals` that will be plotted
using Matplotlib.  __Your task is to fill in the code in this loop so that the array yval contains the frames that are generated by your Monte Carlo simulation.__

The steps that need to be completed in the loop (and which you must code) are as follows:

1. You need to generate a new particle position `newpos`.  To generate this new position you a uniform continuous random variable, `U`, that is between -`maxshift` and +`maxshift` and add it to `pos`.  You thus generate a new position by moving `pos` either left or right by a small (random) ammount.

2. You calculate the energy at the new position `newpos` by calling the function `energy`.  Save this new energy to a variable called `newenergy`

3. If `newenergy` is less or equal to `oldenergy` then you __accept__ the move.  You thus set `pos=newpos` and `oldenergy=newenergy`

4. If `newenergy` is greater than `oldenergy` then you calculate the following ratio:

$$
\tau = \frac{e^{-\beta E_2} }{ e^{-\beta E_1} }
$$

where $E_2$ is `newenergy` and $E_1$ is `oldenergy` and $\beta = \frac{1}{k_B T}$. The variable `temp` contains the value of $k_B T$ that I will assume you have used in your code when I test it.

5. You then generate a second uniform random variable between 0 and 1, `Ua`, if `Ua` is less than the ratio you calculated in step 4 (r) then you accept the move and set `pos=newpos` and `oldenergy=newenergy`.  If it is greater then the move is rejected and no changes are made to the values of `pos` or `oldenergy`.

This 5 step process will then be repeated multiple times because it is in a loop.  The graph that is plotted will thus show the position the particle took in the accepted moves.  Whenever a move is not accepted you will see two consecutive points that have the same y-value.

_N.B. We do a very small number of steps here to ensure as if you a large number of steps the test is more likely to fail.  If you were trying to do a real Monte Carlo calculation for this problem you would most probably run a much longer simulation._

In [ ]:
def energy(x) :
    return x*x / 2

# Set the initial position of the particle, the number of frames,
# the maximum value to shift the position by and the temperature
pos, nframes, maxshift, temp = 0.0, 10, 1.0, 1.0
# Calculate the energy at the start of the simulation
oldenergy = energy(pos)
# Set up some NumPy arrays to hold data
xvals, yvals = np.linspace(1,nframes,nframes), np.zeros(nframes)
for i in range(nframes) :
    # Your code to generate random move goes here
    U = np.random.uniform(-maxshift, maxshift)
    newpos = pos + U

    newenergy = energy(newpos)

    if newenergy <= oldenergy:
      pos = newpos
      oldenergy = newenergy

    else:
      beta = 1/temp
      num = np.exp(-beta*newenergy)
      denom = np.exp(-beta*oldenergy)
      tau = num/denom

      Ua = np.random.uniform(0,1)
      if Ua < tau:
        pos = newpos
        oldenergy = newenergy

    # You need to store the 'time series' of energies in yval to pass the test
    yvals[i] = pos

# This generates the graph
plt.plot( xvals, yvals, 'ko' )
plt.xlabel('index')
plt.ylabel('particle position')
plt.savefig("position.png")
# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_graph4'])

# Writing a fast Monte Carlo code for the Ising model

You are now in a position to write a Monte Carlo for the 2D Ising model by combining what you have learned about evaluating the Ising model Hamiltonian
and what you learned in the previous exercise about how Monte Carlo codes work.  In this code you will:

1. Pick an initial state for the 2D Ising model by generating a 2D NumPy array filled elements that are either -1 or 1.  You will then evaluate the energy of that initial state by using the `hamiltonian` function you wrote for the 6th of the exercises in this notebook.

2. You then write a loop to do the Monte Carlo sampling.  Each time you pass through the loop you generate a trial move by either (a) flipping one spin or (b) flipping all the spins.

3. You then evaluate the energy of the new state that was generated by the trial move and use the accept criteria described in the last exercise to decide if the trial move should be accepted or rejected.

The naive way of implementing the Monte Carlo code is to call the function to evaluate the full Hamiltonian each time you generate a trial move in step 2.  This method of implementing the exercise is not very sensible
as it is computationally expensive.  However, this computational expense can be avoided with a little careful though about the Hamiltonian and our trial moves.  Remember that the Hamiltonian for the 2D Ising model is:

$$
E = - H \sum_i s_i - \sum_{\langle j,j \rangle} s_i s_j
$$

It is useful to express this energy as:

$$
E = - \sum_{i=1}^N \sum_{j=1}^N \frac{s_{i,j}}{2} \left( s_{i-1,j} + s_{i,j-1} + s_{i+1,j} + s_{i,j+1}\right) - H \sum_{i=1}^N \sum_{j=1}^N s_{i,j}
$$

where the double sum ensures we run over all the lattice sites.  Notice that if all the spins are flipped the term in the above sum that describes the interaction does not change as the relative orientations of all pairs of spins
in the system does not change.  The only thing that changes is the interaction with the magnetic field which changes sign.  If all the spins are flipped we can calculate the energy of the new state as follows:

$$
E_2 = E_1 + 2 H \sum_{i=1}^N \sum_{j=1}^N s_{i,j}
$$

where $E_1$ is the energy of the old state.  Notice, furthermore, that when you sum over all the spins you get the magnetiation, $M$.  We can thus obtain the new magnetisation, $M_2$, and the new energy, $E_2$, from the old magnetisation, $M_1$, and old energy, $E_1$ as follows:

$$
M_2 = -1 \times M_1 \qquad \textrm{and} \qquad E_2 = E_1 + 2 \times M_1
$$

Similar tricks can be used to calculate the new energy and magnetisation when the spin on lattice site $(i,j)$ reverses direction as follows:

$$
M_2 = M_1 - 2s_{i,j} \qquad \textrm{and} \qquad E_2 = E_1 + 2s_{i,j}\left( s_{i-1,j} + s_{i,j-1} + s_{i+1,j} + s_{i,j+1} + H \right)
$$

By using these tricks you avoid the computational expense associated with recalculating the full Hamiltonian every time you generate a trial move.

__Your task in this exercise is to implement these tricks yourself and to revise the process of calculating the full Hamiltonian__  You must complete the following two function:

`new_energy` takes in four arguments `spins`, `E`, `H` and `move`.  `spins` is a 2D NumPy array that contains the coordinates of $N^2$ Ising spins and `E` is the energy of the configuration in `spins`.  `H` is the magnetic field strength.  `move` is an integer that is greater than or equal to 0 and less than $N^2 + 1$.  If this integer is equal to $N^2$ your function should return the energy of a configuration in which all the spins point in the opposite direction to the one they point in `spins`.  For all other values of `move` you should determine a single spin to flip using the following piece of python code:

```python
# You will flip the coordinate with spins[i,j] with i and j calculated as follows
N = spins.shape[1]
i, j = int( np.floor( move / N ) ), move%N
```

You should return the energy of the configuration that has this particular spin flipped.

Please use the tricks that I have described above to make your `new_energy` function fast.  You code will fail if you call `hamiltonian` from the function `new_energy`


In [ ]:
def new_energy(spins, E, H, move):
    nrows, ncols = spins.shape
    N = ncols

    # Case 1: Flip all spins
    if move == N**2:
        # Magnetisation of current state
        M1 = np.sum(spins)
        E2 = E + 2 * H * M1
        return E2

    # Case 2: Flip a single spin at (i, j)
    i = int(np.floor(move / N))
    j = move % N

    s = spins[i, j]

    # Periodic neighbours
    up    = spins[(i-1) % nrows, j]
    down  = spins[(i+1) % nrows, j]
    left  = spins[i, (j-1) % ncols]
    right = spins[i, (j+1) % ncols]

    neighbors = up + down + left + right

    E2 = E + 2 * s * (neighbors + H)
    return E2

In [ ]:
runtest(['test_hamiltonian4', 'test_move'])

# Writing a Monte Carlo code

You now have all the pieces that you need in order to write a Monte Carlo code to simulate the 2D Ising model.  In this exercise you are thus going
put all the ideas you have learned together and you are going to write your own Monte Carlo code for simulating the 2D Ising model.

I have written an outline for your code in the following cell, which you need to fill in. As part of this outline I have written a function called
`monte_carlo` to perform Monte Carlo simulations.  At the bottom of the cell I have then used this function to run a short Monte Carlo simulation.
Once you complete the function and run the code a graph will be generated that will show you the energies that were sampled during your Monte Carlo
run.

__Your task is to complete the function `monte_carlo`__.  This function takes the following 7 input arguments:

* `N` - the number of production steps of Monte Carlo to perform.  Statistics are accumulated during the production phase of the run.

* `equil` - the number of equilibration steps that should be performed before starting the production phase of the simulation.  All the states that are visited during this equilibration phase are discarded.  They are not used when accumulating any averages and statistics.

* `stride` - the frequency that should be used for collecting statistics during the production phase of the calculation.  Averages and statistics are only updated every `stride` steps.  States visted on steps that are not a multiple of stride are discarded and not used when accumulating averages and statistics.

* `L` - the size of the sysstem.  An LxL array of spins in simulated.

* `H` - the magnetic field strength

* `T` - the temperature

* `seed` - the random number seed (see the footnote at the end of the instructions for further details).

`monte_carlo` should return a NumPy array with `N/stride` elements.  As you can see from the outline, the elements of this array should be the energies of the states that have been visted during the Monte Carlo simulation.

To complete the function you need to do the following tasks:

1. Write code to calculate the energy of the initial configuration.  The initial configuration is set up in the NumPy array called `spins`.  To calculate the energy of this configuration you will need to use what you learned in the exercises on evaluating these spin hamiltonians.  The energy should be stored in a variable called `eng`.

2. Write code to calculate the energy of the new state that is generated by the trial move.  You will notice that I generate a random integer `move`.  This variable determines the random move that should be performed.  If `move` is equal to `L*L` then the trial move involves flipping every spin.  If `move` is equal to any other value then the `spin[j,k]` is flipped.  You should set the variable `neweng` equal to the energy of the new configuration.  To complete this task you will need to use what you learned in the exercise that came just before this one.

3. You need to decide whether or not to accept the trial move.  You need to complete the if statement that contains a call to the `min` function.  To complete this line of the code you will need to use what you learned in the first exercise on Monte Carlo that you completed.  The exercise where you used Monte Carlo to sample a harmonic potential.

4. You need to update the `spins` array so that it contains the trial move when the trial move is accepted. If this updating of `spins` is not performed then we cannot determine the energy of the trial move when we start the new move on the next pass through the loop.

Once you have added these four bits of code and run the program a graph will be generated that shows the energy that the system adopted over the course of the simulation.  You should see the energy settle down to a constant value and that it then fluctuates around that constant value over the course of the simulation.

===

## On random numbers

The random numbers that computers generate are pseudo random. If you are running Monte Carlo calculations this is usually an annoyance as if you are not careful your results are affected by the fact that the series of
moves that you performed is not really random.  However, when I was writing the code that tests your work this lack of randomness was enormously convnient.  Basically, I test that your Monte Carlo code works corrected by running the same
Monte Carlo calculation you did using the exact same sequence of random numbers that you used in your implementation.  I am able to do this because I pass the variable `seed` to your function.  This variable is then passed to NumPy and is
used to determine how the pseudo random variables it generates are determined.  When I run the Monte Carlo and when you run the Monte Carlo this variable `seed` takes the same value.  The random numbers we generate are thus the same.  The
result you get from your code and the result that I get from my test code are thus identical.


In [ ]:
def monte_carlo( N, equil, stride, L, H, T, seed ) :
    # Set the seed to the value input
    np.random.seed(seed)

    # Generate the initial configuration
    spins = getstate(L)

    # Calculate the energy of the initial configuration that you generated
    eng = hamiltonian(spins, H)

    # Do the main Monte Carlo loop
    neweng, energies = 0, np.zeros( int( np.floor(N / stride) ) )
    for i in range(equil + N) :
        # Generate the trial move
        move = np.floor( (L*L+1)*np.random.uniform(0,1) )
        if move==L*L :
           # Flips all spins
           spins = spins*-1
           neweng = hamiltonian(spins, H)
        else :
           # Flips desired spin
           j, k = int( np.floor( move / L ) ), int( move%L )
           spins[j,k] = spins[j,k]*-1
           neweng = hamiltonian(spins, H)


        e1 = np.exp(-eng/T)
        e2 = np.exp(-neweng/T)
        if np.random.uniform(0,1) < min(1, e2/e1):
           # Accept move -> set the energy to its new value
           eng = neweng

        else:
            # Reject move -> undo the flip(s)
            if move == L*L:
                spins = spins * -1
            else:
                spins[j,k] = spins[j,k] * -1


        step = i-equil
        if step>=0 and step%stride==0 :
            energies[int(step/stride)] = eng

    return energies


# This is the part of the code that allows you to visualise the series of
# energies your Monte Carlo function generates.  You do not need to modify
# anything from here onwards.  You can modify this code though.
energies = monte_carlo( 1000, 0, 10, 20, 0, 1.0, 319 )
plt.plot( energies, 'ko' )
plt.xlabel('index')
plt.ylabel('energy')
plt.savefig("energy.png")
# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_MC1'])

# Estimating the magnetic susceptibility

By completing the last exercise you demonstrate that you can sample states using the Metropolis Monte Carlo algorithm. To analyse the sequence of sampled states that this algorithm generates you can use the same tools that you used in the previous exercise to analyse molecular dynamics trajectories.  In other words, when you take averages over observables for the sampled states you get estimates of ensemble averages.  You can also use block averaging to obtain estimates of the error on these estimates.  In doing all this, however, you need
to take care that you have equilibrated the system before you start collecting statistics.

__In this exercise I want you to estimate a thermodynamic quantity by performing a Monte Carlo simulation__.  The quantity I want you to estimate is the
susceptibility per atom:

$$
\chi = \frac{1}{L^2} \frac{ \langle M^2 \rangle - \langle M \rangle^2}{k_B T}
$$

In this expression $\langle M \rangle$ is the ensemble average of the magnetisation:

$$
M = \sum_{i=1}^L \sum_{j=1}^L s_{i,j}
$$

where $s_{i,j}$ is the spin on site $i,j$ and the sum runs over all the $L^2$ lattice sites.  $\langle M^2 \rangle$ is the average of the squares of magnetisation.

The calculation you are going to perform here is discussed in more detail in the following video.

There are also some notes on this quantity that you can read [here](https://farside.ph.utexas.edu/teaching/329/lectures/node110.html).

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/KDY6WoALyh4?si=Jvr8gs_Y-lS0eHPH" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

I have written an outline code in following cell.  As in the last exercise I have written a function called `monte_carlo` that takes the following 7 input arguments:

* `N` - the number of production steps of Monte Carlo to perform.  Statistics are accumulated during the production phase of the run.

* `equil` - the number of equilibration steps that should be performed before starting the production phase of the simulation.  All the states that are visited during this equilibration phase are discarded.  They are not used when accumulating averages.

* `stride` - the frequency that should be used for collecting statistics during the production phase of the calculation.  Averages are only updated every `stride` steps.  States visted on steps that are not a multiple of stride are discarded and not used when accumulating averages.

* `L` - the size of the sysstem.  An LxL array of spins in simulated.

* `H` - the magnetic field strength

* `T` - the temperature

* `seed` - the random number seed

This function should return one scalar - your estimate for the suceptibility that is computed using the formula above.  To complete the code you will need to complete the following tasks:

1.  Write code to calculate the energy and magnetisation of the initial configuration.  The initial configuration is set up in the NumPy array called `spins`.  To calculate the energy of this configuration you will need to use what you learned in the exercises on evaluating these spin hamiltonians.  The energy should be stored in a variable called `eng`.  You have also learned how to calculate the magnetisation in previous exercises.

2. Write code to calculate the energy of the new state that is generated by the trial move.  You will notice that I generate a random integer `move`.  This variable determines the random move that should be performed.  If `move` is equal to `L*L` then the trial move involves flipping every spin.  If `move` is equal to any other value then the `spin[j,k]` is flipped.  You should set the variable `neweng` equal to the energy of the new configuration.  To complete this task you will need to use what you learned in the exercise that came just before the previous one.

3. You need to decide whether or not to accept the trial move.  You need to complete the if statement that contains a call to the `min` function.  To complete this line of the code you will need to use what you learned in the first exercise on Monte Carlo that you completed.  The exercise where you used Monte Carlo to sample a harmonic potential.

4. You need to update the `spins` array so that it contains the trial move when the trial move is accepted. If this updating of `spins` is not performed then we cannot determine the energy of the trial move when we start the new move on the next pass through the loop.  At this point you also need to determine the new value of the magnetisation.  Please note that you can quickly update the magnetisation from the old magnetisation in much the same way as you have learned to quickly recompute the energy.  You do not (and should not) sum all the elements of the `spins` array on every step

5. You need to add code to accumulate the ensemble averages <M> and <M^2>.  I would use the variables `M` and `M2` to hold the numerators of these ensemble averages and `ns` to hold the denominator.  Please note that to pass the test you should only
modify these varaibles after the first `equil` steps of Monte Carlo have been completed.  Furthermore, even when those first `equil` steps have been complted you should only be updating `M`, `M2` and `ns` every `stride` steps.

6. At the end of the run you should calculate the suscetibility using the formula above and return this value.

Once you have completed those four bits of code and the code is run the values of the suscebility for two distinct two temperatures will be output.

In [ ]:
def monte_carlo( N, equil, stride, L, H, T, seed ) :
    # Set the seed to the value input
    np.random.seed(seed)

    # Generate the initial configuration
    spins = getstate(L)

    # Calculate the energy of the initial configuration that you generated
    # YOUR CODE TO CALCULATE THE INITIAL ENERGY AND THE INITIAL MAGNETISATION OF THE CONFIGURATION GOES HERE
    eng = hamiltonian(spins, H)
    mag = np.sum(spins)

    # Do the main Monte Carlo loop
    neweng, M, M2, ns = 0, mag, 0, 0
    ensM = np.zeros(equil+N)
    ensM2 = np.zeros(equil+N)
    for i in range(equil + N) :
        ns = i
        # Generate the trial move
        move = np.floor( (L*L+1)*np.random.uniform(0,1) )
        if move==L*L :
           # Your code to calculate the energy when all the spins are flipped goes here
           spins = spins*-1
           neweng = hamiltonian(spins, H)
        else :
           # This is going to be flipping a single spin
           j, k = int( np.floor( move / L ) ), int( move%L )
           spins[j,k] = spins[j,k]*-1
           neweng = hamiltonian(spins, H)

        # Now decide whether or not to accept the move
        e1 = np.exp(-eng/T)
        e2 = np.exp(-neweng/T)
        if np.random.uniform(0,1) < min(1, e2/e1):
           # Set the energy to its new value
           eng = neweng
           # Update the spins array
           if move==L*L :
               # YOU NEED TO ADD CODE HERE TO UPDATE THE SPINS AND THE MAGNETISATION
               spins = spins*-1
               M = -M
               M2 = M**2
           else :
               # YOU NEED TO ADD CODE HERE TO UPDATE THE SPINS AND THE MAGNETISATION
               j, k = int( np.floor( move / L ) ), int( move%L )
               spins[j,k] = spins[j,k]*-1
               M -= 2*spins[j,k]
               M2 = M**2

        # YOU NEED TO ADD CODE HERE TO ACCUMULATE YOUR ENSEMBLE AVERAGES
        ensM[i] = M
        ensM2[i] = M2

    ensM = np.sum(ensM)/L**2
    ensM2 = np.sum(ensM2)/L**2




    # YOU SHOULD CALCULATE THE SUSCETIBILITY FROM THE ENSEMBLE AVERAGES HERE
    S = (ensM2 - ensM**2) / (T * L**2)
    return S

# Lets look at the time series of energies from our Monte Carlo simulation
print('The suceptibility at T=1 and H=-1 is', monte_carlo (30, 10, 1, 10, -1, 1, 858) )
print('The suceptibility at T=5 and H=0 is', monte_carlo( 10000, 1000, 10, 20, 0, 5, 450 ) )


In [ ]:
runtest(['test_MC2'])

# Calculating a histogram

By now you should be pretty comfortable using Monte Carlo to estimate ensemble averages.  You have seen that to estimate an ensemble average for an observable, A, you simply calculate
the value of that observable for each microstate that was generated in your Monte Carlo simulation.  You then add all these observations together and divide by the number of observations.
In other words, you calculate the mean for the sample.  Similarly, you have also seen how we can calculate the variance for a sampled observable and how these variances are related to response
functions such as the heat capacity and susceptibility.

For this last batch of exercises I have given some data that has been collected from an Monte Carlo simulation.  You can load this data by executing the command in the cell below.

In [ ]:
# Read in the list of magnetisation values
mags = np.loadtxt('https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/New-MTH4332/IsingModel/magnetisations')

We are going to use this data to estimate the distribution that an observable takes from a Monte Carlo simulation by taking a histogram. We are then going to learn how to calculate the free energy as a function of our observable and how we can report these free energy surfaces (together with suitable errors) in a paper.  The reason this is a useful thing to do is explained in the following video.

You should really know how to calculate a histogram by now but if you need a refresher you can find some notes [here](https://www.notion.so/Histogram-2d2527795f0140008b318d3bc958ee4c) and a detailed tutorial that covers these ideas [here](https://www.plumed-tutorials.org/lessons/21/002/data/GAT_SAFE_README.html).  

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/-1NLaqOJKS0?si=x1cBoivYe2ZNNsv3" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

When you executed the last python code cell before this markdown cell you created a NumPy array called `mags`.  This array contains the magnetisation values for each of the frames that were visited during a short Monte Carlo simulation of a $20 \times 20$ 2D Lattice of Ising spins.  These magnetisations were calculated using:

$$
M = \sum_i s_i
$$

where the sum runs over all the spins and the $s_i$ are the spin coordinates.  __Your task is to draw a histogram that gives an estimate of the probablity density as a function of the average magnetisation
per spin__

$$
\overline{M} = \frac{M}{L^2} = \frac{1}{L^2} \sum_i s_i
$$

Your estimate for the probability density should be normalised so:

$$
\int_{-\infty}^\infty P(\overline{M})\textrm{d}\overline{M} = 1
$$

Furthermore, as the probability density is a continous function, when you draw your graph you should draw a graph with a continuous line showing the estimated distribution with the x-axis label 'average magnetisation per spin' and y-axis label 'probability density'.

To estimate the probability density you should construct a histogram with `nbins` bins.  The first of these bins should start at `minx` and the last of them should end at `maxx`.  The estimate for the pdf that you get from each
of the bins in your histogram should be plotted at the midpoint of the bin.

In [ ]:
nbins = 50
L = 20

magBar = mags / (L**2)

minx = magBar.min()
maxx = magBar.max() + 1e-10  # nudge maxx slightly to avoid boundary index error

edges = np.linspace(minx, maxx, nbins + 1)
width = (maxx - minx) / nbins
midpoints = (edges[:-1] + edges[1:]) / 2

counts, _ = np.histogram(magBar, bins=edges)
total = len(magBar)
pdf = counts / (total * width)

plt.plot(midpoints, pdf)
plt.xlabel('average magnetisation per spin')
plt.ylabel('probability density')
plt.savefig("probDens.pdf")

fighand = plt.gca()

# 1. Check normalisation - should equal 1.0
print("Area under curve:", np.sum(pdf) * width)

# 2. Check midpoints match what the test expects
# The test uses: xb = int(np.floor((mave - minx) / ddd))
# So ddd should equal your width
ddd = (maxx - minx) / nbins
print("bin width:", width)
print("test bin width:", ddd)
print("match:", np.isclose(width, ddd))

# 3. Manually replicate the test's counting logic and compare to your pdf
yv = np.zeros(nbins)
n = 0
for m in mags:
    mave = m / (20 * 20)
    xb = int(np.floor((mave - minx) / ddd))
    yv[xb] = yv[xb] + 1
    n = n + 1

# Normalise the manual counts the same way you do
pdf_manual = yv / (n * width)

print("\nYour pdf vs manual replication:")
print("Max difference:", np.max(np.abs(pdf - pdf_manual)))
print("Are they equal?", np.allclose(pdf, pdf_manual))

In [ ]:
runtest(['test_graph1'])

# Calculating the free energy

If we are running a simulation in the canonical ensemble the probability density that we have just calculated is an estimate for the following integral.

$$
P(\overline{M}) = \frac{1}{Z} \int_{-\infty}^\infty \int_{-\infty}^\infty \dots \int_{-\infty}^\infty \textrm{d}x_1 \textrm{d}x_2 \dots \textrm{d}x_n \delta(\overline{M} - M(\mathbf{x}))e^{-\beta H(\mathbf{x})}
$$

where:

$$
Z = \int_{-\infty}^\infty \int_{-\infty}^\infty \dots \int_{-\infty}^\infty \textrm{d}x_1 \textrm{d}x_2 \dots \textrm{d}x_n e^{-\beta H(\mathbf{x})}
$$

In this expression $\delta$ is a Dirac delta function and the integral here runs over all of phase space. It is thus a multidimensional integral over
multiple coordinates.  It is useful to introduce an energy F(M) of the single coordinate, M, that would have given rise to the distribution above as follows:

$$
P(\overline{M}) = \frac{1}{Z} e^{-\beta F(\overline{M})}
$$

Notice that in introducing this energy we have just used what we know about the canonical ensemble.  The $F(\overline{M})$ that we have introduced here is a quantity known as the free energy.
It can be estimated from the probability density that we obtained from our simulation using:

$$
F(\overline{M}) \propto -k_B T \log\left[ P(\overline{M}) \right]
$$

__Your task in this exercise is to estimate the free energy F(M) as a function of the magnetisation per spin__. You should use the data in the variable `mags` that we read in previously and analyzed in the previous exercise. You should also draw a graph with a continuous line showing the estimated free energy with the x-axis
label 'average magnetisation per spin' and y-axis label 'free energy / natural units'.

To estimate the free energy you should construct a histogram with `nbins` bins.  The first of these bins should start at `minx` and the last of them should end at `maxx`.  The estimate of the free energy that you get from each
of the bins in your histogram should be plotted at the midpoint of the bin.  Please note that I ran the simulation at a temperature equivalent to kT=5.0.

N.B.

When you calculate the free energy from the probability density you will need to ensure that the probablity density is normalised to pass this exercise.  In other words, your estimate of the probablity density, $P(\overline{M})$, should satisfy:

$$
\int_{-\infty}^\infty P(\overline{M})\textrm{d}\overline{M} = 1
$$

When calculating free energies this normalisation is not strictly necessary.  Notice that if we take the first equation in these instructions and insert it in the second equation we arrive at:

$$
F(\overline{M}) = -k_B T \log\left( \int_{-\infty}^\infty \int_{-\infty}^\infty \dots \int_{-\infty}^\infty \textrm{d}x_1 \textrm{d}x_2 \dots \textrm{d}x_n \delta(\overline{M} - M(\mathbf{x}))e^{-\beta H(\mathbf{x})} \right) + k_B T \log( Z )
$$

In other words, the normalisation constant here (which is equal to the partition function) only appears in an additive constant.  We could thus get the free energy from the unormalised histogram.

The fact that the normalisation doesn't matter is important.  Noting this fact reminds us that whenever we quote energies (or free energies) we need to quote them relative to some reference structure.
In other words, every time we give an energy (or free energy) of a state A we are actually giving the difference between the energy of state A and some other (reference) state B.  Your work is incomplete if
you give an energy without specifying the state that has zero energy.

__N.B. You can use the histogram you calculated for the previous exercise to determine the free energy surface.__


In [ ]:





# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_graph2'])

# Error bars for free energy surfaces

As you know any averages that we calculate from Monte Carlo simulations are estimates and must be reported with errors.  The same holds for these estimates
of the free energy that we have just learned to calculate.  In this exercise I am going to show you how to calculate and report these errors.

The first step in calculating the errors comes once you recognise that the fraction of counts in each of the bin in the histogram is an estimate of an ensemble average.  The observable we are calculating in this case is a funtion that is one when $\overline{M}$ is between $a$ and $b$ and zero otherwise.  We can thus use the block averaging
technique that should be familar by now to get multiple estimates for these ensemble averages.  In other words, we split up the trajectory into $N$ blocks.  We then calculate $N$ separate estimates of the histogram from these $N$ blocks of trajectory.  The final histogram and errors upon it that we report are then determined by computing averages and variances from these $N$ separate estimates.  To get the error for a $p_c$ confidence limit from the variance we use:

$$
\epsilon_k = \sqrt{\frac{\sigma_k^2}{N}} \Phi^{-1}\left( \frac{p_c + 1}{2} \right)
$$

where $\sigma_k^2$ is the estimate for the variance, $N$ is the number of blocks and $\Phi^{-1}$ is the inverse of the cumulative distribution function for a standard normal distribution.
This last function can be calculated using:

```python
scipy.stats.norm.ppf( (pc+1)/2 )
```

Notice that when we calculate a histogram we are calculating $M$ ensemble averages in one shot, where $M$ is the number of bins we have used when calculating our histogram.  At the end of the process of calculating the histogram we will thus have $M$ averages, which we will call $p_k$, and $M$ error values, which we will call $e_k$.  As you know the free energy is obtained from the histogram using:

$$
f_k = -k_B T \log p_k
$$

The error on the free energy is thus obtained as:

$$
e(f)_k = k_B T \frac{e_k}{p_k}
$$

When I draw an estimate for a free energy surface in one of my papers I like the width of the line I draw to indicate the confidence limit on my estimate of the free energy surface.  I will
thus use a command like this one:

```python
plt.fill_between( xv, f - e, f + e )
```

which generates the figure shown below:

![](https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/New-MTH4332/IsingModel/fillbetween_example.png)

In the python fragment above the NumPy array `xv` contains the mid points of the histogram bins.  `f` is a NumPy array that contains the estimates of the free energy that were obtained using block averaing.
`e` is the error on the estimate of the free energy.  Hopefully, you can see how any line through the filled part of the curve is consistent with the results I obtained.

__Your task for this exercise is to generate a plot of the free energy surface with error bars calculated in the way described above__  You should analyse the data in the NumPy array `mags` that we created at the start of this section on calculating free energy.  You should also use blocks with size `nblocks` in your program.

The data I have given you was collected at a temperature that corresponded to $k_B T = 5$ and is (obviously) the same as the data you have had in the previous two exercises.  As with the previous two exercises, to estimate the free energy you should construct a histogram with `nbins` bins.  The first of these bins should start at `minx` and the last of them should end at `maxx`.  The estimate of the free energy that you get from each of the bins in your histogram should be plotted at the midpoint of the bin.

The tests don't look at the graph you are generating but instead look at the values in the two arrays `lower_yv` and `upper_yv`.  These arrays must be set so that a graph similar to the one shown above is drawn with the
shaded area representing the 90% confidence limit on your estimate of the free energy.

Notice that as in the previous exercise you need to ensure that you normalise the histograms before computing free energies to pass the tests.

=====

A final note of caution: You will find multiple bins in the histogram which will have zero visits in the trajectory. The error for these bins should also be zero but will come out as nan (not a number) as in calculating the error we need to divide by the fraction of time spent in the bin (which is zero). $a/0$ cannot be computed which is why you get nan.

I mention this because, to pass the test, you will need to ensure that there are no nan values in the arrays `lower_yv` and `upper_yv` that are plotted.  You can get rid of these values by ensuring that you do not divide by zero.  In other words, you will need to calculate the error for a bin differently when the estimate for the histogram in that bin is zero and when the estimate is non-zero.


In [ ]:
# This is the number of bins
nbins = 50
# This is the minimum and maximum for the grid
minx, maxx = -1.1, 1.1
# These variables should hold the x coordinates of the graph and the upper and lower
# confident limits on the estimate of the free energy
xv, lower_yv, upper_yv = np.zeros( nbins ), np.zeros( nbins ), np.zeros( nbins )

# This is the size of the blocks.  You will calculate one estimate of the histogram
# over the blocks of this size
blocksize = 200
# This is the number of blocks that you are splitting the trajectory in
nblocks = int( np.floor( len(mags) / blocksize ) )
# This loop calculates your nblocks estimates of the histogram
for i in range(nblocks) :
    # Your code for calculating each of the nblocks estimate of the histogram goes here



# This part plots the graph.  You need to define and set the variables lower_yv and upper_yv
# as described in the instructions
plt.fill_between( xv, lower_yv, upper_yv )
plt.xlabel('average magnetisation per spin')
plt.ylabel('free energy / natural units')
plt.savefig("fes.png")

In [ ]:
runtest(['test_x', 'test_errors'])

# Ensuring errors are not underestimated

Hopefully you found extending concepts of block averaging that you learned about when you were estimating ensemble averages to estimating free energy surfaces not too difficult.  This procedure was covered in the last exercise.  There is one further concept that you learned about in the earlier exercises about block averaging that we need to cover here.  Once we have covered that in this exercise you can move on to the assignment.

As you hopefully remember we introduced block averaging because we were worried about correlations between random variables extracted from trajectories.  The existence of these correlations ensures that we will underestimate the variance if we calculate it the normal way.  We thus do this business with blocks instead.

When you last looked at calculating block averages you learned to draw graphs that showed how the variance changes with block size like this one:

![](https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/New-MTH4332/IsingModel/block-average1.png)

For this exercise you need to do something similar and investigate how the size of the average error for the histogram depends on the sizes of the blocks that it is computed from. You should calculate the block-averaged histograms for block sizes of 10, 20, 25, 100, 200, 250 and 500 along with the errors on the average histograms.  From these histograms and errors you can caluclate the free energy and its associated error.  If you then take an average of the errors for those bins that have been visted more than 0 times you can get a measure of the average
error on the estimate of the free energy for each block size.  You will thus be able to produce a graph something like the one shown below:

![](https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/New-MTH4332/IsingModel/block-average2.png)

It is this graph that is tested at the end of the exercise.  As always you need to use the data in the NumPy array called `mags` to produce your graph.  This array contains data from a Monte Carlo trajectory that was run at a tmperature equivalant to $k_BT=5$.

You should construct a histogram with `nbins` bins.  The first of these bins should start at `minx` and the last of them should end at `maxx`.  Your graph should have estimates of the average for block sizes of
10, 20, 25, 100, 200, 250 and 500. The x-axis label should be 'Length of block' the y axis label should be 'Average error on free energy'.

In [ ]:
# This is the number of bins
nbins = 50
# This is the minimum and maximum for the grid
minx, maxx = -1.1, 1.1
# This is the list of block sizes you should investigate
blocksizes = np.array([10, 20, 25, 100, 200, 250, 500])


# Your code to draw the graph described in the instructions goes here.
# I would recommend writing a function to calculate the histogram with
# different block sizes

# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_graph3'])